<a href="https://colab.research.google.com/github/Chiaradisanto/Segmentation/blob/main/FinalCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###  **Save DICOM FILES as .png images**



Connect to Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')



Importing libraries


In [ ]:
%matplotlib inline
import numpy as np # linear algebra
import pandas as pd 
import os
import scipy.ndimage
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
from  scipy import ndimage



Installing and importing module for reading and writing DICOM files

In [ ]:
!pip install pydicom
import pydicom
from pydicom import dcmread


**get_names** function takes as input the path of the selected subject containing the DICOM files 



In [ ]:
def get_names(path):
    names = []
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            _, ext = os.path.splitext(filename)
            if ext in ['.dcm']:
                names.append(filename)
    return names

DICOM files are ordered with sort function. 

In [ ]:
names = sorted(get_names(path))

A list containig all the DICOM files is created. 
This operation loads all DICOM images from the folder into a list for manipulation.

In [ ]:
files = []
for x in names:
        files.append(pydicom.dcmread(path+'/'+x))
len(files)


Make sure that all the files contains SliceLocation in it. Otherwhise skip them



In [ ]:
slices = []
skipcount = 0
for f in files:
    if hasattr(f, 'SliceLocation'):
        slices.append(f)
    else:
        skipcount = skipcount + 1

print("skipped, no SliceLocation: {}".format(skipcount))

Slices are sorted according to SliceLocation

In [ ]:
slices = sorted(slices, key=lambda s: s.SliceLocation, reverse=True)  #reverse is True to sort in descending order


The voxel values in the images are raw. 

**dicom_HU** converts raw values into Houndsfeld units.
This function takes as input all the slices of considered subject.
The transformation is linear. 

Both the rescale intercept and rescale slope are stored in the DICOM header at the time of image acquisition.
The final value is rescaled to HU.
Windowing is then applied in order to adjust the grayscale level of the images.




In [ ]:
def dicom_HU(scan):
    image = np.stack([s.pixel_array*s.RescaleSlope+s.RescaleIntercept for s in scan],axis=2).astype(np.int16) #

    img_min = 120 - 120 // 2 #minimum HU level
    img_max = 120 + 120 // 2 #maximum HU level
    window_image = image.copy()

    window_image[window_image < img_min] = img_min #set img_min for all HU levels less than minimum HU level
    window_image[window_image > img_max] = img_max #set img_max for all HU levels higher than maximum HU level
    plt.figure(figsize=(20, 10))
  
    plt.style.use('grayscale') 
    plt.imshow(window_image[:,:,0], cmap='gray')
    plt.axis('off')
    return  np.array(window_image)

Save as images in png format in a specific folder. A folder for each patient was created.

In [ ]:
num_slices=len(files)
for i in range(1,num_slices):
    scan=dicom_HU(slices[i-1:i])
    plt.axis('off')
    plt.savefig(f"{images_path}"+str(i)+".png", bbox_inches='tight',pad_inches = 0,dpi=300, quality=95) #dpi represents the resolution in dots per inch.
                                                                                                        # A high value results in a high resolution image.
                                                                                                    
                                                                                                        #quality represents the image quality
                                                                                                        #on a scale from 1 (worst) to 95 (best).
    plt.show()

These operations must be repeated for all the subjects changing the paths.